In [ ]:
import read_mta_stations as s
import read_mta_turnstile as t
import read_tech_companies as c
import create_station_heatmap as h

Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180505.txt
Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180512.txt
Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180519.txt
Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180526.txt


In [5]:
stations = s.read_mta_stations()
stations.sample(5)

,key,name,LINE,lon,lat
242,243,Aqueduct - North Conduit Av,A,-73.83405799948723,40.668234001699815
200,201,Pelham Bay Park,6-6 Express,-73.82812100059289,40.85246199951662
415,416,City Hall,R-W,-74.00698581780337,40.71327233111697
162,163,81st St,A-B-C,-73.97209794937208,40.78134608418206
169,170,116th St,2-3,-73.94962500096905,40.802097999133004
